# Calculate ERA5 vertically integrated moisture flux

Calculates the vertically integrated moisture flux for ERA5 atmospheric river calculations

1. Read in u, v, q data for NDJFM
    * Input: `/glade/work/mkorendy/metisregimes/era5.AR.variable_pna_NDJFM.1986-2016.nc`
2. Get moisture flux uq, vq at different levels
3. Integrate moisture flux over pressure
4. Read the data out
    * Output: `/glade/work/mkorendy/metisregimes/era5.AR.vqvi_pna_NDJFM.1986-2016.nc`

## Imports

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

## Define constants

In [3]:
# Gravity, scaled by pressure change
g = .0981

# Interpolation? (native, N32, or N128)
interpolate = 'N128'

## Files

In [4]:
#Path
path = '/glade/work/mkorendy/metisregimes/'

#Files
if interpolate == 'N32':
    f_u = path + 'era5.AR.u_pna_NDJFM.1986-2016.nc'
    f_v = path + 'era5.AR.v_pna_NDJFM.1986-2016.nc'
    f_q = path + 'era5.AR.q_pna_NDJFM.1986-2016.nc'
elif interpolate == 'native':
    f_u = path + 'era5.AR.u_pna_NDJFM.1986-2016.reg_grid.nc'
    f_v = path + 'era5.AR.v_pna_NDJFM.1986-2016.reg_grid.nc'
    f_q = path + 'era5.AR.q_pna_NDJFM.1986-2016.reg_grid.nc'
else:
    f_u = path + 'era5.AR.u_pna_NDJFM.1986-2016.N128.nc'
    f_v = path + 'era5.AR.v_pna_NDJFM.1986-2016.N128.nc'
    f_q = path + 'era5.AR.q_pna_NDJFM.1986-2016.N128.nc'

#f_u2 = path + 'era5.AR.u_pna_NDJFM.2016.nc'
#f_v2 = path + 'era5.AR.v_pna_NDJFM.2016.nc'
#f_q2 = path + 'era5.AR.q_pna_NDJFM.2016.nc'

#Output
if interpolate == 'N32':
    f_out = path + 'era5.AR.vqvi_pna_NDJFM.1986-2016.nc'
elif interpolate == 'native':
    f_out = path + 'era5.AR.vqvi_pna_NDJFM.1986-2016.reg_grid.nc'
else:
    f_out = path + 'era5.AR.vqvi_pna_NDJFM.1986-2016.N128.nc'
#f_out = path + 'era5.AR.vqvi_pna_NDJFM.2016.nc'

In [5]:
print(f_out)

/glade/work/mkorendy/metisregimes/era5.AR.vqvi_pna_NDJFM.1986-2016.N128.nc


## Read data in

In [6]:
ds_u = xr.open_dataset(f_u).drop(['number','step','valid_time'])
ds_v = xr.open_dataset(f_v).drop(['number','step','valid_time'])
ds_q = xr.open_dataset(f_q).drop(['number','step','valid_time'])

#ds_u2 = xr.open_dataset(f_u2).drop(['number','step','valid_time'])

In [7]:
ds_u = ds_u.reindex(time=sorted(ds_u['time'].values))
ds_v = ds_v.reindex(time=sorted(ds_u['time'].values))
ds_q = ds_q.reindex(time=sorted(ds_u['time'].values))

In [8]:
ds_u

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 14, lat: 86, lon: 213, time: 9074)
Coordinates:
  * time           (time) datetime64[ns] 1986-01-01 ... 2015-12-31T12:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 925.0 850.0 ... 50.0 30.0 10.0
  * lat            (lat) float64 20.0 20.7 21.4 22.11 ... 78.25 78.95 79.65
  * lon            (lon) float64 150.5 151.2 151.9 152.6 ... 298.1 298.8 299.5
Data variables:
    u              (time, isobaricInhPa, lat, lon) float32 ...

## Get moisture flux

In [9]:
da_uq = ds_u['u'] * ds_q['q']
da_vq = ds_v['v'] * ds_q['q']

KeyboardInterrupt: 

## Integrate moisture flux over pressure

In [8]:
#Define level pressures
pressures = [1000-962.5, 962.5 - 887.5, 887.5 - 775, 100, 100, 100, 100, 100, 100, 100, 150-75, 75-40, 40-20, 400-350]
len(pressures)

14

In [9]:
levs = da_uq['isobaricInhPa']
uqvi = []
vqvi = []

for k in np.arange(0,len(pressures)-1):
    uqvi.append(da_uq.sel(isobaricInhPa = levs[k]) * (pressures[k])/g)
    vqvi.append(da_vq.sel(isobaricInhPa = levs[k]) * (pressures[k])/g)

In [10]:
ds_uqvi = xr.combine_nested(uqvi, concat_dim='levs')
ds_vqvi = xr.combine_nested(vqvi, concat_dim='levs')

In [11]:
# Sum over pressure levels
da_uqvi = ds_uqvi.sum('levs')
da_vqvi = ds_vqvi.sum('levs')

In [12]:
np.max(da_vqvi.values.flatten())

1611.2743

## Merge data into a dataset

In [13]:
ds_uqvi = da_uqvi.to_dataset(name = 'uqvi')
ds_vqvi = da_vqvi.to_dataset(name = 'vqvi')

In [14]:
ds = xr.merge([ds_uqvi,ds_vqvi])

## Read data out

In [15]:
ds.to_netcdf(f_out)